### Import packages

In [1]:
import json
import pandas as pd
import requests
from bookstack_api_framework.api_token_secret import *
from bookstack_api_framework.shelves import Shelves
from bookstack_api_framework.pages import Pages
from bookstack_api_framework.filter import *
from bookstack_api_framework.api_url import *
from bookstack_api_framework.bookstack_api import BookStackAPI

### Set up BookStack API framework

In [2]:
api = BookStackAPI(base_url, token_id, token_secret)

### Get books and pages on specified shelf

In [3]:
#Function returning list of a specified BookStack record type, featuring pagination so that all records are 
#in the output, not just the first 100 from one single API call
def get_all_items(item):
    possibilities = ['attachments', 'books', 'chapters', 'pages', 'shelves', 'users']
    if item not in possibilities:
        return ("Invalid argument. Please try using one of the following options: " + str(possibilities))
    all_records = []
    offset = 0
    api_url = (base_url + '/' + item)
    base_response = requests.get(api_url, headers={'Authorization': 'Token {0}:{1}'.format(token_id, token_secret)})
    base_response_json = base_response.json()
    all_records.append(base_response_json['data'])
    while len(all_records) < base_response_json['total']:
        offset += 100
        new_api_url = (api_url + '/?offset=' + str(offset))
        new_response = requests.get(new_api_url, headers={'Authorization': 'Token {0}:{1}'.format(token_id, token_secret)})
        new_response_json = new_response.json()
        all_records.append(new_response_json['data'])
    return all_records

#Get books on specified shelf
def get_book_ids_from_shelf(shelf_id):
    shelf = api.shelves.read(id=shelf_id)
    books_on_shelf = shelf['books']
    book_id_list = []
    for book in books_on_shelf:
        book_id_list.append(book['id'])
    return book_id_list

#Function to get all pages of books from specified shelf
def get_page_ids_from_books(book_ids):
    all_pages = get_all_items('pages')
    page_id_list = []
    for dict in all_pages[0]:
        if dict['book_id'] in book_ids:
            page_id_list.append(dict['id'])
    return page_id_list

#First, get book ids from specified shelf
shelf_id = 1
book_ids = get_book_ids_from_shelf(shelf_id)
#Second, get page ids
page_ids = get_page_ids_from_books(book_ids)

### Analyze unique tags & unique tag values by tag

In [4]:
#Functions to get unique tags and values
def get_unique_tags(page_ids):
    unique_tag_list = []
    for page_id in page_ids:
        page_info = api.pages.read(id=page_id)
        for tag in page_info['tags']:
            if tag['name'] not in unique_tag_list:
                unique_tag_list.append(tag['name'])
    return '\n'.join(sorted(unique_tag_list))

def read_values_from_tag(tag):
    values_from_tag = []
    read_values = tag['value'].split("|")
    for el in read_values:
        el = el.strip()
        if len(el) > 0:
            values_from_tag.append(el)
    return values_from_tag

def get_unique_values(page_ids, target_tag):
    values_list = []
    for page_id in page_ids:
        page_info = api.pages.read(id=page_id)
        for tag in page_info['tags']:
            if tag['name'] == target_tag:
                values_list += read_values_from_tag(tag)
    unique_value_list = list(dict.fromkeys(values_list))
    return '\n'.join(sorted(unique_value_list))

In [5]:
#Apply get_unique_tags function to analyze unique tags of pages from books on specified shelf
print(get_unique_tags(page_ids))

Angebot
GPS
Kategorie
Kategorien
Online-Shop


In [6]:
#Apply get_unique_values function to analyze unique tags in tag "Kategorie"
print(get_unique_values(page_ids, target_tag='Kategorie'))

Accessoires
Acessoires
Alkoholfreier Sekt
Aufstriche
Backwaren
Bar
Bio-Speisen
Bäckerei
Cafe
Café
Concept Store
Eier
Elektronische Geräte
Eventcatering
Eventlocation
Foodsharing
Gemüse
Geschenkartikel
Getränke
Gewürze
Haushalt
Haushalt & Kosmetik
Haushaltswaren
Honig
Hygiene
Hülsenfrüchte
Inklusion
Kaffe
Kaffeebar
Kartoffeln
Katzenartikel
Kleidung
Kleinmöbel
Kosmetik
Kosmetik & Haushalt
Kunsthandwerk
Körperhygiene
Körperpflege
Laden
Lebensmittel
Lebensmitttel
Lieferdienst
Markt
Mehle
Milchprodukte
Naschereien
Naturkosmetik
Nüsse
Obst
Rehaverein
Reinigungsmittel
Reis
Restaurant
Schuhe
Second Hand
Sekt
Selbstbedienung
Selbstgemachte Nudeln
Shop
Shopo
Speisen zum Mitnehmen
Spielzeug
Spot
Tee
Teigwaren
Textilien
Tierprodukte
Trockenfrüchte
Unverpackt
Weine
Öl & Essig


In [7]:
#Apply get_unique_values function to analyze unique tags in tag "Angebot"
print(get_unique_values(page_ids, target_tag='Angebot'))

Abendrobe
Accessoires
Alkoholfreier Sekt
Alkoholfreier Wein
Anzüge
Auflagen
Aufstriche
Baby
Baby Mode
Babynahrung
Backwaren
Bad
Badezusätze
Bar
Barten
Bekleidung
Bett
Bettwäsche
Bio-Lebensmittel
Blusen
Brot
Bussinessoutfits
Bücher
Büro
Cafe
Catering
Cider
Cremes
Cupcakes
Damen
Damen & Herren
Damen und Herren
Damenhygiene
Deko
Diffusoren
Eier
Essig
Essig & Öl
Essig und Öl
Events
Ferttiggerichte
Filet
Fisch- und Fleischalternativen
Fleisch
Frühstück
Futterbaum
Gemüse
Geschenke
Geschenksets
Gesichtspflege
Getreide
Getränke
Gewürze
Glas
Grundnahrungsmittel
Gulasch
Handtaschen
Haushalt
Heimtextilien
Heiße Küche
Hemden
Herren
Holzdeko
Home goods
Honig
Hoodies
Hosen
Hygiene
Hülsenfrüchte
Imbiss
Jacken
Jakcen
Jeans
Jungen
Kaffee
Kaffee & Tee
Kaffee und Kuchen
Kaffee und Trinkschokolade
Kartoffeln
Katzenmöbel
Kaugummis
Kids
Kinder
Kleider
Kleidung
Kosmetik
Kuchen
Kunstpostkarten
Käse
Kümmel-Brand
Lampen
Lebensmittel
Limonaden
Matratzen
Mehl
Men
Milch
Milchproduktalternativen
Milchprodukte
Mitgl